In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cv2

import sys
import time

sys.path.insert(1, '../../Src/')

from utils import vision
from utils import camera

In [2]:
from models.yolov8face import model_yolo8face

In [3]:

# Initialize YOLOv8_face object detector
YOLOv8_face_detector = model_yolo8face.YOLOv8_face("yolov8n-face.onnx")
#YOLOv8_face_detector = model_yolo8face.YOLOv8_face("yolov8-lite-s.onnx")
#YOLOv8_face_detector = model_yolo8face.YOLOv8_face("yolov8-lite-t.onnx")

In [4]:


srcimg = cv2.imread('../../Src/images/1.jpg')

# Detect Objects
boxes, scores, classids, kpts = YOLOv8_face_detector.detect(srcimg)

# Draw detections
dstimg = YOLOv8_face_detector.draw_detections(srcimg, boxes, scores, kpts)
#cv2.imwrite('result.jpg', dstimg)
winName = 'Deep learning face detection use OpenCV'
cv2.namedWindow(winName, 0)
cv2.imshow(winName, dstimg)
cv2.waitKey(0)
cv2.destroyAllWindows()

[ WARN:0@0.169] global net_impl.cpp:174 setUpNet DNN module was not built with CUDA backend; switching to CPU


In [5]:
boxes, scores

(array([[213.90427244, 184.70634498, 244.92698821, 329.72636751]]),
 array([0.8468767], dtype=float32))

In [6]:
COLOR_BOUNDINGBOX = (255, 0, 0)
THICKNESS_BOUNDINGBOX = 2


# For webcam input:
#cap = cv2.VideoCapture(0)
cap = camera.ParallelCamera(2).start()


start webcam


In [7]:


my_fps = vision.FPS(100)

my_fps_model = vision.FPS(100)


while True:
  my_fps.start()

  # Read image
  success, image = cap.read()  
  # Flip the image horizontally for a selfie-view display.    
  image = cv2.flip(image.copy(), 1)

  image = cv2.resize(image,(300,300))
  #time.sleep(0.5)

  if not success:
    print("Ignoring empty camera frame.")
    continue

  my_fps_model.start()
  
  # Model Inference
  image.flags.writeable = False
  #predictions = model.predict(image)
  boxes, scores = YOLOv8_face_detector.predict(image)
  image.flags.writeable = True  
  
  # Draw bounding boxes
  image = vision.draw_boundingboxes(image,boxes,COLOR_BOUNDINGBOX,THICKNESS_BOUNDINGBOX)
  
  # Calculate FPS
  my_fps_model.update()
  my_fps.update()
  # Show image
  image = vision.put_text(image,"FPS      : "+str(np.round(my_fps.fps(),3)),pos =(20,20))
  image = vision.put_text(image,"FPS model: "+str(np.round(my_fps_model.fps(),3)),pos =(20,50))
  cv2.imshow('Face Detection Project',image)
 
  # Close window with ESC
  if cv2.waitKey(5) & 0xFF == 27:
    break

# DestroyWindows
cap.release()
cv2.destroyAllWindows()


In [8]:
boxes

[[121.63348732866325,
  172.09099868054182,
  179.62408804130064,
  265.52373129378793]]